# Prepare, index, then search

## 1. Data generation

Use this code in your solutions to generate the dataset.

In [1]:
%%time
def build(N, D):
    dataset = [None] * N
    for i in range(N):
        dataset[i] = [((i % 9997 - d) + (i * d - d)) % 9999 for d in range(D)]
        dataset[i] = tuple(dataset[i])
    return dataset
DATASET = build(100000, 3)

Wall time: 92.3 ms


Here we check they are unique.

In [2]:
from collections import Counter
Counter(DATASET).most_common(10)

[((0, 9997, 9995), 1),
 ((1, 0, 9998), 1),
 ((2, 2, 2), 1),
 ((3, 4, 5), 1),
 ((4, 6, 8), 1),
 ((5, 8, 11), 1),
 ((6, 10, 14), 1),
 ((7, 12, 17), 1),
 ((8, 14, 20), 1),
 ((9, 16, 23), 1)]

## 2. Search with indexing

Implement search using the following indices:
- [Annoy](https://github.com/spotify/annoy). Build 5+ trees (take more if 5 is not enough) with Euclidean distance.
- Scipy [kd-tree](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) implementation. Euclidean distance in query.

All vectors are unique. You are given a query: a method of search (`annoy`|`kdtree`) and a vector which is exactly present in a dataset (3D integer vector). Print **index** of this vector to `output.txt`.

E.g. `input.txt`
```
annoy
0 9997 9995
```
then `output.txt` will be:
```
0
```

Example of Annoy:

In [22]:
%%time
from annoy import AnnoyIndex
from scipy.spatial import KDTree

n_trees = 7
dim = 3
with open('input.txt', 'r') as file:
    method, vector = file.read().split('\n')
v = tuple(map(int, a.split(' ')))
Following the examples from https://github.com/spotify/annoy
if method == 'annoy':
    t = AnnoyIndex(dim, 'euclidean')
    for i in range(DATASET.legth):
        t.add_item(i, DATASET[i])
    t.build(n_trees)
    result = t.get_nns_by_vector(v, 1, include_distances=False)[0]
    with open('output.txt', 'w+') as o:
        o.write(str(result))
if method == 'kd-tree':
    # Using only Scipy documentation
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.query.html
    t = KDTree(DATASET)
    d, result = t.query(x=v, p=2)
    print(result)
    with open('output.txt', 'w+') as o:
        o.write(str(result))
    
# ... for [0, 9997, 9995] returns 0 in 1.37s with index building

2
Wall time: 114 ms
